# XFEAt Screw dislocation

Create a screw dislocation in a bcc Fe crystal and apply a shear stress until the
dislocation starts moving.

Author: Alexander Hartmaier  
Institution: ICAMS / Ruhr University Bochum, Germany  
Date: March 2022

This work is licensed under a
Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International License
<a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/">(CC-BY-NC-SA)</a>

The XFEAt package comes with ABSOLUTELY NO WARRANTY. This is free
software, and you are welcome to redistribute it under the conditions of
the GNU General Public License <a href="http://www.fsf.org/licensing/licenses/gpl.html">(GPLv3)</a>

<img alt="Creative Commons License" style="border-width:0;max-heigt:9px;height:100%;" src="https://i.creativecommons.org/l/by-nc-sa/4.0/88x31.png"/></a>
<img alt="GPLv3" style="border-width:0;max-heigt:9px;height:100%;" src="../docs/figures/gplv3-88x31.png"/></a>

In [1]:
%matplotlib inline
from pyvista import set_plot_theme, set_jupyter_backend
set_plot_theme('document')
set_jupyter_backend('ipyvtklink')

## 1. Create model

Define a material and create an XFEAt model consisting of an atomistic core and an XFEM frame around it. Insert a screw disloction into the XFEM and atomistic parts of the model.

In [2]:
import xfeat

# define material of single crystal as dictionary
mat = {
       'name' : 'iron',
       'cs'   : 'bcc',  # crystal structure
       'lp'   : 2.8553,  # lattice parameter in Angstrom
       'mass' : 55.845,  # atomic mass in u
       # define anisotropic elastic constants in GPa
       'C11'  : 243.4,
       'C12'  : 145.0,
       'C44'  : 116.0,
       # W-Bop 6
       # C11. C12, C44 = 2.837825, 1.5317, 0.6256225
       # define crystallograhic orientation of crystal
       'ori_x' : [-1, 2, -1],
       'ori_y' : [-1, 0,  1],
       'ori_z' : [ 1, 1,  1],
       }

# create XFEM model 
mod = xfeat.Model(mat, size=400)
# create atomic core
mod.atoms([10, 17, 3])
mod.atom_grid.plot(cpos='xy', style='points', point_size=20, render_points_as_spheres=True)
# create mesh and set up system stiffness matrix
mod.mesh()
mod.grid.plot(cpos='xy', show_edges=True)
# create screw dislocation
mod.init_dislo([0, 0, 1])
# extract atomistic displacements
mod.atom_bc()
# plot z-displaceent of atoms and XFEM nodes
mod.plot('uz')

Succesfully created structure with 6120 atoms.


ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

Generated mesh with 17376 elements and 23872 nodes around atomistic core.


ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

XFEM solution obtained in 9.7519s.
Relaxation of atoms obtained in 13.919s.

Created dislocation with Burgers vector ([0. 0. 1.]) in model.
Norm of Burgers vector is 2.472772 A


ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

It is seen that the z-displacement in atomistic and XFEM regions are constistently defined when the screw dislocation is introduced. However, the other displacement components are depending of the interatomic potential and cannot be set directly.

In [3]:
# plot y-displacements
mod.plot('uy')
print('Zoom in to see atoms lying in overlap region more clearly.')

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

Zoom in to see atoms lying in overlap region more clearly.


To embedd the dislocation consistently into the XFEM model, it is necessary to relax both structures iteratively. During this process, the atoms overlapping with the XFEM region are always fixed to the XFEM solution, whereas the atoms in the inner region are relaxed freely. After the atomic relaxation has occured, the displacements of the relaxed atoms on the boundary of the inner region are applied as XFEM boundary conditions and a new XFEM solution is calculated.

In [4]:
# iterate into relaxed configuration
for i in range(12):
    mod.atom_bc()  # apply relaxed atom positions as BC to XFEM 
    mod.solve()  # colculate nodal displacements for mechanical equilibrium
    mod.shift_atoms()  # move boundary atoms in overlap region according to XFEM strain field
    mod.relax_atoms(i)  # relax atomic structure with fixed boundary atoms
    if i % 3 == 2:
        print(f'y-displacement after {i+1} iterations. Zoom in to overlap region.')
        mod.plot('uy')

XFEM solution obtained in  9.321s.
Relaxation of atoms obtained in 17.499s.
XFEM solution obtained in 9.8902s.
Relaxation of atoms obtained in 16.039s.
XFEM solution obtained in 9.5095s.
Relaxation of atoms obtained in 18.124s.
y-displacement after 3 iterations. Zoom in to overlap region.


ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

XFEM solution obtained in 9.7734s.
Relaxation of atoms obtained in 16.729s.
XFEM solution obtained in 9.6368s.
Relaxation of atoms obtained in 15.448s.
XFEM solution obtained in 9.7709s.
Relaxation of atoms obtained in  16.98s.
y-displacement after 6 iterations. Zoom in to overlap region.


ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

XFEM solution obtained in 9.6388s.
Relaxation of atoms obtained in 14.403s.
XFEM solution obtained in 9.6008s.
Relaxation of atoms obtained in 14.473s.
XFEM solution obtained in 9.3976s.
Relaxation of atoms obtained in  14.77s.
y-displacement after 9 iterations. Zoom in to overlap region.


ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

XFEM solution obtained in  9.648s.
Relaxation of atoms obtained in 14.585s.
XFEM solution obtained in 9.3361s.
Relaxation of atoms obtained in 13.792s.
XFEM solution obtained in 9.4643s.
Relaxation of atoms obtained in 15.357s.
y-displacement after 12 iterations. Zoom in to overlap region.


ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

## 2. Apply shear stress
In a first step, a sub-critical shear stress is applied at the XFEM boundary, which is not sufficient to move the dislocation.

In [5]:
# plot nodal displacement
print('z-displacement before applying shear stress on XFEM model.')
mod.plot('uz')
# plot stresses
print('Stress component yz before applying shear stress.')
mod.plot('sigyz')
# plot potential energy of atoms
print('Potential energy of atoms before applying shear stress.')
mod.plot('epot')

# Apply sub-critical shear stress on boundary
val = 1.55
mod.apply_bc(val, bc_type='stress', comp='yz')
print(f'Shear stress of {val} GPa applied on XFEM boundary.')
# iterate into relaxed configuration
for i in range(10):
    mod.atom_bc()
    mod.solve()
    mod.shift_atoms()
    mod.relax_atoms(i, name='applied_stress_155')
    if i % 5 == 4:
        print(f'Potential energy of atoms after {i+1} iterations.')
        mod.plot('epot')
# plot stresses
print(f'Stress component yz at applied shear stress {val} GPa.')
mod.plot('sigyz')

z-displacement before applying shear stress on XFEM model.


ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

Stress component yz before applying shear stress.


ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

Potential energy of atoms before applying shear stress.


...

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

XFEM solution obtained in 9.3296s.
Relaxation of atoms obtained in 18.851s.
XFEM solution obtained in 9.3337s.
Relaxation of atoms obtained in 18.423s.
XFEM solution obtained in 9.3519s.
Relaxation of atoms obtained in 17.896s.
XFEM solution obtained in 9.6866s.
Relaxation of atoms obtained in 15.541s.
XFEM solution obtained in 9.7379s.
Relaxation of atoms obtained in 15.824s.
Potential energy of atoms after 10 iterations.


ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

Stress component yz at applied shear stress 1.55 GPa.


ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

## 3. Increase stress 
Now, the level of the applied shear stress is increased beyond the critical value and dislocation motion sets in.

In [6]:
# Apply critical shear stress on boundary
val = 1.65
mod.apply_bc(val, bc_type='stress', comp='yz')
# iterate into relaxed configuration
for i in range(10):
    mod.atom_bc()
    mod.solve()
    mod.shift_atoms()
    mod.relax_atoms(i, name='applied_stress_165')
    if i % 5 == 4:
        print(f'Potential energy of atoms after {i+1} iterations.')
        mod.plot('epot')

# plot stresses
print(f'Stress component yz at applied shear stress {val} GPa.')
mod.plot('sigyz')


XFEM solution obtained in 9.8556s.
Relaxation of atoms obtained in 20.014s.
XFEM solution obtained in 9.9316s.
Relaxation of atoms obtained in 20.502s.
XFEM solution obtained in 9.7549s.
Relaxation of atoms obtained in 20.368s.
XFEM solution obtained in 9.8926s.
Relaxation of atoms obtained in 19.254s.
XFEM solution obtained in 9.8435s.
Relaxation of atoms obtained in 18.676s.
Potential energy of atoms after 5 iterations.


ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

XFEM solution obtained in 9.5456s.
Relaxation of atoms obtained in  20.68s.
XFEM solution obtained in 9.9984s.
Relaxation of atoms obtained in 33.765s.
XFEM solution obtained in 9.6299s.
Relaxation of atoms obtained in 34.382s.
XFEM solution obtained in 9.9793s.
Relaxation of atoms obtained in 35.245s.
XFEM solution obtained in 9.3917s.
Relaxation of atoms obtained in 15.682s.
Potential energy of atoms after 10 iterations.


ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

Stress component yz at applied shear stress 1.65 GPa.


ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

It is seen that the dislocation starts moving at this stress level, which also increases the yz-stress component at the rhs of the inner boundary of the XFEM disproportionately.